In [33]:
import pandas as pd
from sql import get_data
import warnings
warnings.filterwarnings("ignore")
import psycopg2 # Needed to get database errors when uploading dataframe
import sqlalchemy
import nltk  
import string
from nltk.corpus import stopwords
import os
import psycopg2 # Needed to get database errors when uploading dataframe
import sql
import sqlalchemy

In [ ]:
nltk.download('stopwords')

In [2]:
clean_jobs = get_data('SELECT * FROM capstone.jobs_translated')

In [3]:
jobs_new = clean_jobs[['id','job_id', 'posted_date', 'company_name', 'job_title_en2', 'job_description', 'city', 'state', 'country']]

In [4]:
jobs_new['job_title_en'] = jobs_new['job_title_en2']

In [5]:
jobs_complete = jobs_new[['id','job_id', 'posted_date', 'company_name', 'job_title_en', 'job_description', 'city', 'state', 'country']]

In [ ]:
jobs_complete.head()

In [10]:
region_category = {
    'Europe' : ['Ukraine', 'France', 'Spain', 'Sweden', 'Norway', 'Germany', 'Finland', 'Poland', 'Italy', 'United Kingdom', 
                'Romania', 'Belarus', 'Kazakhstan', 'Greece', 'Bulgaria', 'Iceland', 'Hungary', 'Portugal', 'Serbia', 'Austria', 
                'Czech Republic', 'Ireland', 'Lithuania', 'Latvia', 'Croatia', 'Bosnia and Herzegovina', 'Slovakia', 'Estonia', 
                'Denmark', 'Switzerland', 'Netherlands', 'Moldova', 'Belgium', 'Albania', 'North Macedonia', 'Slovenia', 
                'Montenegro', 'Kosovo', 'Azerbaijan', 'Luxembourg', 'Andorra', 'Malta', 'Liechtenstein', 'San Marino', 
                'Monaco', 'Vatican City', 'Cyprus',  'Czechia', 'Czech Republic'],

    'Africa' : ['Nigeria', 'Ethiopia', 'Egypt', 'DR Congo', 'Tanzania', 'South Africa', 'Kenya', 'Uganda', 'Algeria', 
                'Sudan', 'Morocco', 'Angola', 'Mozambique', 'Ghana', 'Madagascar', 'Cameroon', 'Niger', 'Burkina Faso', 'Mali', 
                'Malawi', 'Zambia', 'Senegal', 'Chad', 'Somalia', 'Zimbabwe', 'Guinea', 'Rwanda', 'Benin', 'Burundi', 'Tunisia', 
                'South Sudan', 'Togo', 'Sierra Leone', 'Libya', 'Congo', 'Liberia', 'Central African Republic', 'Mauritania', 'Eritrea', 
                'Namibia', 'Gambia', 'Botswana', 'Gabon', 'Lesotho', 'Guinea-Bissau', 'Equatorial Guinea', 'Mauritius', 'Eswatini', 'Djibouti'],

'Middle East' : ['Turkey', 'Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Yemen', 'Abkhazia', 'Armenia', 
                'Artsakh', 'Azerbaijan', 'Georgia', 'South Ossetia', 'Iraq', 'Israel', 'Jordan', 'Lebanon', 'Palestine', 'Syria', 'Iran', 
                'Akrotiri and Dhekelia', 'Cyprus'],
                
'North America' : ['USA', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 
                'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
                'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
                'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
                'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
                'West Virginia', 'Wisconsin', 'Wyoming', 'Canada', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 
                'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 
                'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'],

'Central America' : ['Mexico', 'Guatemala', 'Cuba', 'Haiti', 'Dominican Republic', 'Honduras', 'Nicaragua', 'El Salvador', 'Costa Rica', 'Panama', 
                    'Jamaica', 'Trinidad and Tobago', 'Belize', 'Bahamas', 'Barbados', 'Saint Lucia', 'Grenada', 'Antigua and Barbuda', 'Dominica', 
                    'Saint Kitts and Nevis'],

'South America' : ['Argentina', 'Bolivia', 'Bouvet Island', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Falkland Islands', 'French Guiana', 
                'Guyana', 'Paraguay', 'Peru', 'South Georgia', 'Suriname', 'Uruguay', 'Venezuela', 'South Sandwich Islands'],

'South Asia' : ['Afghanistan', 'Bangladesh', 'Bhutan', 'India', 'Nepal', 'Maldives', 'Pakistan', 'Sri Lanka'],

'Southeast Asia' : ['Brunei', 'Cambodia', 'East Timor', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],

'Central Asia' : ['Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan'],

'East Asia' : ['China', 'Hong Kong', 'Macau', 'Japan', 'Mongolia', 'North Korea', 'South Korea', 'Taiwan']

}

In [28]:
jobs_complete = jobs_complete.drop(columns='job_title_trans', axis=1)

In [7]:
jobs_complete['search_country'] = jobs_complete['country']

In [19]:
world_regions = pd.Series([])

for row in range(len(jobs_complete)):

    for key in region_category:

        for value in region_category[key]:
            
            if jobs_complete['country'][row] == value:
                world_regions[row] = key
   
jobs_complete.insert(9, "world_region", world_regions)

In [21]:
jobs_complete['job_title_trans'] = jobs_complete['job_title_en']

In [30]:
jobs_complete['posted_date'] = pd.to_datetime(jobs_complete['posted_date'], format="%Y-%m-%d")